## CrewAI核心组件 - Crew

+ 使用YAML配置
+ 使用代码配置
+ Crew的输出

### 使用YAML配置来创建Crew

In [ ]:
from crewai import Agent, Crew, Task, Process
from crewai.project import CrewBase, agent, task, crew, before_kickoff, after_kickoff


@CrewBase
class YourCrewName:
    """你的团队描述"""

    # YAML配置文件的路径
    # 要查看YAML中定义的代理和任务示例，请查看以下内容:
    # - 任务:https://docs.crewai.com/concepts/tasks#yaml-configuration-recommended
    # - 代理:https://docs.crewai.com/concepts/agents#yaml-configuration-recommended
    agents_config = "config/agents.yaml"
    tasks_config = "config/tasks.yaml"

    @before_kickoff
    def prepare_inputs(self, inputs):
        # 在团队开始前修改输入
        inputs["additional_data"] = "一些额外信息"
        return inputs

    @after_kickoff
    def process_output(self, output):
        # 在团队完成后修改输出
        output.raw += "\n启动后处理完成。"
        return output

    @agent
    def agent_one(self) -> Agent:
        return Agent(config=self.agents_config["agent_one"], verbose=True)

    @agent
    def agent_two(self) -> Agent:
        return Agent(config=self.agents_config["agent_two"], verbose=True)

    @task
    def task_one(self) -> Task:
        return Task(config=self.tasks_config["task_one"])

    @task
    def task_two(self) -> Task:
        return Task(config=self.tasks_config["task_two"])

    @crew
    def crew(self) -> Crew:
        return Crew(
            agents=self.agents,  # 由@agent装饰器自动收集
            tasks=self.tasks,  # 由@task装饰器自动收集
            process=Process.sequential,
            verbose=True,
        )


### 使用代码进行创建

In [ ]:
from crewai import Agent, Crew, Task, Process
from crewai_tools import YourCustomTool  # 假设你有一个自定义工具


class YourCrewName:
    def agent_one(self) -> Agent:
        return Agent(
            role="数据分析师",
            goal="分析市场数据趋势",
            backstory="一位有经济学背景的经验丰富的数据分析师",
            verbose=True,
            tools=[YourCustomTool()],
        )

    def agent_two(self) -> Agent:
        return Agent(
            role="市场研究员",
            goal="收集市场动态信息",
            backstory="一位勤奋且具有敏锐洞察力的研究员",
            verbose=True,
        )

        def task_one(self) -> Task:
            return Task(
                description="收集最近的市场数据并识别趋势。",
                expected_output="一份总结市场关键趋势的报告。",
                agent=self.agent_one(),
            )

        def task_two(self) -> Task:
            return Task(
                description="研究影响市场动态的因素。",
                expected_output="一份关于影响市场因素的分析。",
                agent=self.agent_two(),
            )

        def crew(self) -> Crew:
            return Crew(
                agents=[self.agent_one(), self.agent_two()],
                tasks=[self.task_one(), self.task_two()],
                process=Process.sequential,
                verbose=True,
            )

### Crew的输出

In [ ]:
crew = Crew(
    agents=[research_agent, writer_agent],
    tasks=[research_task, write_article_task],
    verbose=True,
)

# 使用kickoff方法启动团队
crew_output = crew.kickoff()

# 打印输出
# 打印输出属性
print(f"'Raw Output: {crew_output.raw}")

if crew_output.json_dict:
    print(f"JSON Output:{json.dumps(crew_output.json_dict, indent=2)}")
if crew_output.pydantic:
    print(f"Pydantic Output:{crew_output.pydantic}")

print(f"Tasks Output:{crew_output.tasks_output}")
print(f"Token Usage: {crew_output.token_usage}")